In [1]:
import copy
from typing import cast

from build123d import Polygon, extrude, Pos, Compound, Box
from ocp_tessellate.utils import Color
from jupyter_cadquery import show, open_viewer, set_defaults

from kfactory import LayerEnum

from gdsfactory.component import Component
from gdsfactory.technology import DerivedLayer, LayerStack, LayerViews, LogicalLayer
from gdsfactory.typings import LayerSpecs

import gdsfactory as gf

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [2]:
cv = open_viewer("GDS")

In [3]:
INSTANCES = {}

In [10]:
def _get_extruded_polygon(polygon, height, zmin, color, decimals=4, optimize=True):
    if optimize and len(polygon) == 4:

        dx0 = round(polygon[3][0] - polygon[0][0], decimals)
        dx1 = round(polygon[2][0] - polygon[1][0], decimals)
        dy0 = round(polygon[1][1] - polygon[0][1], decimals)
        dy1 = round(polygon[2][1] - polygon[3][1], decimals)
        h = round(height, decimals)

        if dx0 == dx1 and dy0 == dy1:
            # The viewer supports instances, so render a box only once and
            # reference the instance moved to the right position

            if INSTANCES.get((dx0, dy0, h)) is None:
                INSTANCES[(dx0, dx1, h)] = Box(dx0, dy0, h)
                reference = INSTANCES[(dx0, dx1, h)]
            else:
                reference = copy.copy(INSTANCES[(dx0, dx1, h)])

            center = (polygon[0][0] + dx0 / 2, polygon[0][1] + dy0 / 2)
            obj = Pos(*center, zmin + height / 2) * reference
            obj.color = Color(color)
            return obj

    p = Polygon(*polygon, align=None)
    obj = extrude(p, amount=-height)
    obj = Pos(0, 0, zmin) * obj
    obj.color = Color(color)
    return obj


def to_b123d(
    component: Component,
    layer_views: LayerViews | None = None,
    layer_stack: LayerStack | None = None,
    exclude_layers: LayerSpecs | None = None,
) -> Compound:
    """Return build123d Compound.

    Args:
        component: to extrude in 3D.
        layer_views: layer colors from Klayout Layer Properties file.
            Defaults to active PDK.layer_views.
        layer_stack: contains thickness and zmin for each layer.
            Defaults to active PDK.layer_stack.
        exclude_layers: list of layer index to exclude.

    """
    from gdsfactory.pdk import (
        get_active_pdk,
        get_layer,
        get_layer_stack,
        get_layer_views,
    )

    try:
        from trimesh.creation import extrude_polygon
        from trimesh.scene import Scene
    except ImportError as e:
        print("you need to `pip install trimesh`")
        raise e

    layer_views = layer_views or get_layer_views()
    layer_stack = layer_stack or get_layer_stack()

    exclude_layers = exclude_layers or ()
    exclude_layers = [get_layer(layer) for layer in exclude_layers]

    component_with_booleans = layer_stack.get_component_with_derived_layers(component)
    polygons_per_layer = component_with_booleans.get_polygons_points(
        merge=True,
    )
    has_polygons = False

    sub_assemblies = []
    assembly = Compound(label="GDS")
    for level in layer_stack.layers.values():
        layer = level.layer

        if isinstance(layer, LogicalLayer):
            assert isinstance(layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(layer.layer))
        elif isinstance(layer, DerivedLayer):
            assert level.derived_layer is not None
            assert isinstance(level.derived_layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(level.derived_layer.layer))
        else:
            raise ValueError(f"Layer {layer!r} is not a DerivedLayer or LogicalLayer")

        layer_index = int(get_layer(layer_tuple))

        if layer_index in exclude_layers:
            continue

        if layer_index not in polygons_per_layer:
            continue

        zmin = level.zmin
        layer_view = layer_views.get_from_tuple(layer_tuple)
        assert layer_view.fill_color is not None
        if zmin is not None and layer_view.visible:
            has_polygons = True
            polygons = polygons_per_layer[layer_index]
            height = level.thickness
            objects = []
            sub_assembly = Compound(label=str(f"{layer_view.name} ({round(zmin,4)})"))
            print(layer_view.name, len(polygons))
            for polygon in polygons:
                if layer_view.name == "M3":
                    print(height, polygon)
                obj = _get_extruded_polygon(
                    polygon,
                    height,
                    zmin,
                    layer_view.fill_color.as_rgb_tuple(),
                    optimize=True,
                )
                objects.append(obj)
            sub_assembly.children = objects

        sub_assemblies.append(sub_assembly)
    assembly.children = sub_assemblies

    if not has_polygons:
        raise ValueError(
            f"{component.name!r} does not have polygons defined in the "
            f"layer_stack or layer_views for the active Pdk {get_active_pdk().name!r}"
        )
    return assembly

In [11]:
c = gf.c.straight_heater_doped_rib(length=100)

In [12]:
compound = to_b123d(c)
show(compound)

Waveguide 1
SLAB90 1
VIAC 128
M1 2
VIA1 330
M2 2
VIA2 480
M3 2
2.0 [[ -0.2   -25.   ]
 [ -0.2   -15.   ]
 [ 23.133 -15.   ]
 [ 23.133   5.   ]
 [ 33.133   5.   ]
 [ 33.133 -15.   ]
 [ 69.799 -15.   ]
 [ 69.799   5.   ]
 [ 79.799   5.   ]
 [ 79.799 -15.   ]
 [ 89.8   -15.   ]
 [ 89.8   -25.   ]]
2.0 [[-0.2   -5.   ]
 [-0.2   25.   ]
 [89.8   25.   ]
 [89.8   15.   ]
 [56.466 15.   ]
 [56.466 -5.   ]
 [46.466 -5.   ]
 [46.466 15.   ]
 [ 9.8   15.   ]
 [ 9.8   -5.   ]]
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
c = gf.components.dbr(
    w1=0.45,
    w2=0.55,
    l1=0.159,
    l2=0.159,
    n=10,
    cross_section="strip",
    straight_length=0.01,
).copy()
c.draw_ports()

In [14]:
c = gf.components.grating_coupler_elliptical_trenches()

In [20]:
scene = c.to_3d()
scene.show()

In [19]:
c = gf.components.grating_coupler_array(grating_coupler='grating_coupler_elliptical', pitch=127, n=6, port_name='o1', rotation=-90, with_loopback=False, cross_section='strip', straight_to_grating_spacing=10, centered=True).copy()
c.draw_ports()